In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
cardamom=pd.read_csv('/kaggle/input/indian-small-cardamom-price-history-csv/Indian Small Cardamom Price History.csv')

In [ ]:
cardamom.head(5)

In [ ]:
list(filter(lambda x: x.isnumeric()==False,cardamom['Sno']))

In [ ]:
cardamom['Sno'][0]='1'

In [ ]:
cardamom['Sno']=cardamom['Sno'].astype(int)

In [ ]:
cardamom.info()

In [ ]:
from datetime import date
cardamom['Date of Auction']=cardamom['Date of Auction'].str.split('-')
cardamom['Date']=None
cardamom['month']=None
for i in cardamom['Sno']:
    cardamom['Date'][i-1]=cardamom['Date of Auction'][i-1][0]
    cardamom['month'][i-1]=cardamom['Date of Auction'][i-1][1]
cardamom['Date of Auction'][0][0]
# cardamom['date']=cardamom['Date of Auction'][0]
# cardamom['Month']=cardamom['Date of Auction'][1]

In [ ]:
cardamom

In [ ]:
cardamom['Date']=cardamom['Date'].astype(int)

In [ ]:
import datetime
str(cardamom['Date of Auction'][19][1])
datetime.datetime.strptime(str(cardamom['Date of Auction'][19][1]), "%b").month

In [ ]:
import datetime
for i in range(2836):
    cardamom['Date of Auction'][i] = datetime.date(int('20'+(str(cardamom['Date of Auction'][i][2]))), 
                                                         datetime.datetime.strptime(str(cardamom['Date of Auction'][i][1]), "%b").month,
                                                         int(cardamom['Date of Auction'][i][0]))

In [ ]:
cardamom.info()

In [ ]:
cardamom

In [ ]:
cardamom['Date of Auction']=pd.to_datetime(cardamom['Date of Auction'])

In [ ]:
cardamom.info()

In [ ]:
cdf=cardamom[['Sno', 'Date of Auction', 'Auctioneer', 'No.of Lots',
       'Total Qty Arrived (Kgs)', 'Qty Sold (Kgs)', 'MaxPrice (Rs./Kg)',
       'Avg.Price (Rs./Kg)','Date','month']]

In [ ]:
cdf=cdf.rename(columns={'Date of Auction':'dateofauction', 'No.of Lots':'nooflots',
       'Total Qty Arrived (Kgs)':'totalquantityarrived', 'Qty Sold (Kgs)':'sold', 'MaxPrice (Rs./Kg)':'maxprice',
       'Avg.Price (Rs./Kg)':'avgprice'},inplace=False)

In [ ]:
cdf=cdf[cdf.sold!=max(cdf.sold)]

In [ ]:
cdf=cdf

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
cdf.columns

In [ ]:
cdf.describe()

In [ ]:
plt.scatter(cdf.nooflots,cdf.sold,color='blue')
plt.xlabel("nooflots")
plt.ylabel('sold')
plt.show()

In [ ]:
cdf.columns

In [ ]:
x=cdf.drop(columns=['sold','maxprice','avgprice','Sno','dateofauction'])
y=cdf['sold']

In [ ]:
x.columns

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
def encod(x):
    char = x.select_dtypes(["O"]).columns
    le = LabelEncoder()
    for i in x[char]:
        x[i] = le.fit_transform(x[i])

    scale =StandardScaler()
    x = scale.fit_transform(x)
    return x

In [ ]:
x=encod(x)

In [ ]:
# jaadu=[]
# for i in range(100):
#     x_train,x_test,y_train,y_test=train_test_split(x,y, test_size=0.2,random_state=i)
#     lr=LinearRegression()
#     # pipe=make_pipeline((Scale,x),lr)
#     # pipe.fit(x_train,y_train)
#     lr.fit(x_train,y_train)
#     xpred=lr.predict(x_test)
#     jaadu.append(r2_score(y_test,xpred ) )
# print(jaadu)
# print(max(jaadu))
# print(jaadu.index(max(jaadu)))

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y, test_size=0.2,random_state=88)

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
lr=LinearRegression()
lr.fit(x_train,y_train)

In [ ]:
xpred=lr.predict(x_test)

In [ ]:
from sklearn.metrics import r2_score
print("R2-score: %.2f" % r2_score(y_test,xpred ) )

In [ ]:
cardamom['predicted']=cdf.avgprice.shift(-1)

In [ ]:
cardamom.describe()

In [ ]:
cardamom=cardamom.sort_values(by=['Date of Auction'])

In [ ]:
cardamom=cardamom.dropna()

In [ ]:
cardamom

In [ ]:
a_train,a_test,b_train,b_test=train_test_split(np.array([cardamom['Avg.Price (Rs./Kg)']]).T,np.array(cardamom['predicted']),test_size=0.2)

In [ ]:
# li=[]
# for i in range(2,50):
#     from sklearn.ensemble import RandomForestRegressor
#     model=RandomForestRegressor(n_estimators=i, random_state=1)
#     model.fit(a_train, b_train)
#     li.append([i, model.score(a_test,b_test)])
# max(li)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model=RandomForestRegressor(n_estimators=49, random_state=1)
model.fit(a_train, b_train)

In [ ]:
a_pred=model.predict(a_test)

In [ ]:
b_test

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
r2_score(b_test,a_pred)

In [ ]:
p=cardamom.tail(1)
l=np.array(p['Avg.Price (Rs./Kg)'])
l

In [ ]:
# # auctioneer=input('Enter the Auctioneer name: ')
# # Nooflots=input('Enter No of lots : ')
# # totalquantity=input('Enter total quantity arrived:')
# # da=input('Enter the date(dd): ')
# # mon=input('Enter month(mm): ')
# # s=pd.DataFrame(np.array([[auctioneer,Nooflots,totalquantity,da,mon]]),columns= ['Auctioneer', 'nooflots', 'totalquantityarrived', 'Date', 'month'])    
# s=pd.DataFrame([['South Indian Green Cardamom Company Ltd, Kochi',231,78534.2,21,'May']],columns= ['Auctioneer', 'nooflots', 'totalquantityarrived', 'Date', 'month'])
# s=encod(s)
# print(lr.predict(s))


WORK IN PROGRESS .......******